In [ ]:
!pip install langchain_community
!pip install langchain
!pip install faiss-cpu
!pip install langchain_openai

In [1]:
import os
from kaggle_secrets import UserSecretsClient
user_secrets = UserSecretsClient()
secret_value_0 = user_secrets.get_secret("OPENAI_API_KEY")

os.environ["OPENAI_API_KEY"]=secret_value_0

In [2]:
from langchain_community.document_loaders import WebBaseLoader
from langchain_community.vectorstores import FAISS
from langchain_openai import OpenAIEmbeddings
from langchain_text_splitters import RecursiveCharacterTextSplitter

loader = WebBaseLoader("https://docs.smith.langchain.com/overview")
docs = loader.load()
documents = RecursiveCharacterTextSplitter(
    chunk_size=1000, chunk_overlap=200
).split_documents(docs)
vector = FAISS.from_documents(documents, OpenAIEmbeddings())
retriever = vector.as_retriever()

In [4]:
retriever.invoke("데이터셋 업로드는 어떻게 하나요?")[0]

Document(metadata={'source': 'https://docs.smith.langchain.com/overview', 'title': 'Get started with LangSmith | 🦜️🛠️ LangSmith', 'description': 'LangSmith is a platform for building production-grade LLM applications. It allows you to closely monitor and evaluate your application, so you can ship quickly and with confidence. Use of LangChain is not necessary - LangSmith works on its own!', 'language': 'en'}, page_content="Skip to main contentGo to API DocsSearchRegionUSEUGo to AppQuick startTutorialsHow-to guidesConceptsReferencePricingSelf-hostingLangGraph CloudQuick startOn this pageGet started with LangSmithLangSmith is a platform for building production-grade LLM applications. It allows you to closely monitor and evaluate your application, so you can ship quickly and with confidence. Use of LangChain is not necessary - LangSmith works on its own!1. Install LangSmith\u200bPythonTypeScriptpip install -U langsmithyarn add langsmith2. Create an API key\u200bTo create an API key head to

In [23]:
from langchain.tools.retriever import create_retriever_tool
from langchain.tools import tool
from langchain.utilities import SerpAPIWrapper
from langchain.chains import LLMMathChain

retriever_tool = create_retriever_tool(
    retriever,
    "langsmith_search", #표현할 이름
    "Search for information about LangSmith. For any questions about LangSmith, you must use this tool!",)

@tool("math")
def math(query : str) -> str:
    """useful for when you need to answer questions about math"""
    llm_math_chain = LLMMathChain(llm=llm, verbose=True)
    return llm_math_chain.run(query)

#참조 https://api.python.langchain.com/en/latest/tools/langchain_core.tools.create_retriever_tool.html
# retriever (BaseRetriever) –
# name (str) –
# description (str) –
# document_prompt (Optional[BasePromptTemplate]) –
# document_separator (str) –

tools = [retriever_tool, math]

In [24]:
from langchain_openai import ChatOpenAI

llm = ChatOpenAI(model="gpt-4o-mini", temperature=0)

In [25]:
from langchain.prompts import PromptTemplate, MessagesPlaceholder, ChatPromptTemplate,SystemMessagePromptTemplate,HumanMessagePromptTemplate

# 각 템플릿을 생성합니다
system_prompt = SystemMessagePromptTemplate(prompt=PromptTemplate(input_variables=[], template='You are a helpful assistant'))
human_prompt = HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['input'], template='{input}'))
chat_history_placeholder = MessagesPlaceholder(variable_name='chat_history', optional=True)
agent_scratchpad_placeholder = MessagesPlaceholder(variable_name='agent_scratchpad')

# 이들을 ChatPromptTemplate에 통합합니다
chat_prompt = ChatPromptTemplate.from_messages([
    system_prompt,
    chat_history_placeholder,
    human_prompt,
    agent_scratchpad_placeholder
])

# 생성된 chat_prompt를 이제 사용할 수 있습니다.

In [26]:
from langchain.agents import create_tool_calling_agent

agent = create_tool_calling_agent(llm, tools, chat_prompt)

In [27]:
from langchain.agents import AgentExecutor

agent_executor = AgentExecutor(agent=agent, tools=tools, verbose=True)

In [28]:
agent_executor.invoke({"input": "hi!. 500 * 300은 뭐야?"})



> Entering new AgentExecutor chain...

Invoking: `math` with `{'query': '500 * 300'}`




> Entering new LLMMathChain chain...
500 * 300

/opt/conda/lib/python3.10/site-packages/langchain/chains/llm_math/base.py:56: UserWarning: Directly instantiating an LLMMathChain with an llm is deprecated. Please instantiate with llm_chain argument or using the from_llm class method.
  warnings.warn(
/opt/conda/lib/python3.10/site-packages/langchain_core/_api/deprecation.py:151: LangChainDeprecationWarning: The method `Chain.run` was deprecated in langchain 0.1.0 and will be removed in 1.0. Use invoke instead.
  warn_deprecated(


```text
500 * 300
```
...numexpr.evaluate("500 * 300")...

Answer: 150000
> Finished chain.
Answer: 150000500 * 300은 150,000입니다.

> Finished chain.


{'input': 'hi!. 500 * 300은 뭐야?', 'output': '500 * 300은 150,000입니다.'}

In [29]:
agent_executor.invoke({"input": "랑스미스가 뭐야?"})



> Entering new AgentExecutor chain...

Invoking: `langsmith_search` with `{'query': '랑스미스'}`


Get started with LangSmith | 🦜️🛠️ LangSmith

Skip to main contentGo to API DocsSearchRegionUSEUGo to AppQuick startTutorialsHow-to guidesConceptsReferencePricingSelf-hostingLangGraph CloudQuick startOn this pageGet started with LangSmithLangSmith is a platform for building production-grade LLM applications. It allows you to closely monitor and evaluate your application, so you can ship quickly and with confidence. Use of LangChain is not necessary - LangSmith works on its own!1. Install LangSmith​PythonTypeScriptpip install -U langsmithyarn add langsmith2. Create an API key​To create an API key head to the Settings page. Then click Create API Key.3. Set up your environment​Shellexport LANGCHAIN_TRACING_V2=trueexport LANGCHAIN_API_KEY=<your-api-key># The below examples use the OpenAI API, though it's not necessary in generalexport OPENAI_API_KEY=<your-openai-api-key>4. Log your first trace​T

{'input': '랑스미스가 뭐야?',
 'output': '랑스미스(LangSmith)는 프로덕션 등급의 LLM(대형 언어 모델) 애플리케이션을 구축하기 위한 플랫폼입니다. 이 플랫폼은 애플리케이션을 면밀히 모니터링하고 평가할 수 있는 기능을 제공하여, 빠르고 자신 있게 배포할 수 있도록 돕습니다. 랑스미스는 LangChain과 함께 사용할 필요는 없으며, 독립적으로 작동합니다.\n\n주요 기능으로는 API 키 생성, 환경 설정, 트레이스 로깅 등이 있으며, 다양한 방법으로 애플리케이션의 성능을 평가할 수 있는 도구를 제공합니다.'}